### Sample program for 1D CNN  

#### Import libraries  

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from numpy import dstack

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.utils import to_categorical

2021-11-12 09:44:31.391417: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-12 09:44:31.391443: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


#### Functions  

##### load a single file as a numpy array

In [2]:
def load_file(filepath):
    df = pd.read_csv(filepath, delimiter='\s+',
                     skiprows=0, header=None)
    return df.values

##### load a list of files and return as a 3d numpy array

In [3]:
def load_group(filenames, prefix=''):
    loaded = []
    for name in filenames:
        data = load_file(prefix + name)
        loaded.append(data)
    # stack group so that features are the 3rd dimension
    loaded = np.dstack(loaded)
    print(loaded.shape)  # debug
    return loaded

##### load a dataset group, such as train or test

In [4]:
def load_dataset_group(group, prefix=''):
    filepath = prefix + group + '/Inertial_Signals/'
    # load all 9 files as a single array
    filenames = []
    # total acceleration
    filenames += ['total_acc_x_'+group+'.txt', 'total_acc_y_'+group+'.txt', 'total_acc_z_'+group+'.txt']
    # body acceleration
    filenames += ['body_acc_x_'+group+'.txt', 'body_acc_y_'+group+'.txt', 'body_acc_z_'+group+'.txt']
    # body gyroscope
    filenames += ['body_gyro_x_'+group+'.txt', 'body_gyro_y_'+group+'.txt', 'body_gyro_z_'+group+'.txt']
    # load input data
    X = load_group(filenames, filepath)
    # load class output
    y = load_file(prefix + group + '/y_'+group+'.txt')
    return X, y

##### load the dataset, returns train and test X and y elements

In [5]:
def load_dataset(prefix=''):
    # load all train
    trainX, trainy = load_dataset_group('train', prefix + 'HARDataset_small/')
    print(trainX.shape, trainy.shape)
    # load all test
    testX, testy = load_dataset_group('test', prefix + 'HARDataset_small/')
    print(testX.shape, testy.shape)
    # zero-offset class values
    trainy = trainy - 1
    testy = testy - 1
    # one hot encode y
    trainy = to_categorical(trainy)
    testy = to_categorical(testy)
    print(trainX.shape, trainy.shape, testX.shape, testy.shape)
    return trainX, trainy, testX, testy

##### fit and evaluate a model

In [6]:
def evaluate_model(trainX, trainy, testX, testy):
    n_epochs = 10
    batch_size = 32
    n_timesteps = trainX.shape[1]
    n_features = trainX.shape[2]
    n_outputs = trainy.shape[1]
    
    model = Sequential()
    model.add(Conv1D(64, 3, activation='relu',
              input_shape=(n_timesteps,n_features)))
    model.add(Conv1D(64, 3, activation='relu'))
    model.add(Dropout(0.5))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(100, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam', metrics=['accuracy'])
    # exec training
    model.fit(trainX, trainy, epochs=n_epochs,
              batch_size=batch_size, verbose=0)
    
    # evaluate model
    loss, accuracy = model.evaluate(testX, testy, verbose=0)
    return accuracy

##### summarize scores  

In [7]:
def summarize_results(scores):
    m, s = np.mean(scores), np.std(scores)
    print('Ave. accuracy: {:.3f}(+/-{:.3f})%'.format(m, s))

##### run an experiment

In [8]:
def run_experiment(repeats=5):
    # load data
    trainX, trainy, testX, testy = load_dataset()
    # repeat experiment
    scores = []
    for r in range(repeats):
        score = evaluate_model(trainX, trainy, testX, testy)
        score = score * 100.0
        print('#{}: accuracy: {:.3f}%'.format(r+1, score))
        scores.append(score)
    # summarize results
    summarize_results(scores)

#### Execution

In [9]:
%%time
run_experiment()

(735, 128, 9)
(735, 128, 9) (735, 1)
(294, 128, 9)
(294, 128, 9) (294, 1)
(735, 128, 9) (735, 6) (294, 128, 9) (294, 6)


2021-11-12 09:44:35.615048: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-11-12 09:44:35.615073: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-11-12 09:44:35.615086: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (anhvu): /proc/driver/nvidia/version does not exist
2021-11-12 09:44:35.615440: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-12 09:44:35.782756: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR 

#1: accuracy: 81.293%
#2: accuracy: 80.272%
#3: accuracy: 53.401%
#4: accuracy: 56.122%
#5: accuracy: 64.966%
Ave. accuracy: 67.211(+/-11.727)%
CPU times: user 55 s, sys: 1.97 s, total: 56.9 s
Wall time: 12.8 s
